# Minnesota BMSB Analysis: `Preprocessing`
##### Contributors: *Luke Zaruba*, *Mattie Gisselbeck*
##### Last Updated: 2023-04-20

In this notebook, an automated ETL pipeline is implemented to process and clean data for analysis. Before running this notebook, the `extract_data.ps1` script should be run to download several of the datasets needed.

In [1]:
# Import Packages
import os
import sys

import arcgis
import arcpy
import pandas as pd

# Append Path & Import ETL Module
sys.path.append("..")

from bmsb.etl import WeatherLoader, ObservationLoader

In [2]:
# Set up Workspace
gdb_directory = r"C:\gitFiles\minnesota-bmsb-analysis\data\gdb"
gdb_name = "bmsb_analysis"
full_gdb_path = os.path.join(gdb_directory, f"{gdb_name}.gdb")

# Check if GDB Folder Exists
if not os.path.exists(gdb_directory):
    os.mkdir(gdb_directory)
    arcpy.management.CreateFileGDB(gdb_directory, gdb_name)
    
    print("Directory & GDB created")

# Check if GDB Exists (if folder exists)
else:
    if not os.path.exists(full_gdb_path):
        arcpy.management.CreateFileGDB(gdb_directory, gdb_name)
        
        print("Directory exists, GDB created")
        
    else:
        print("Directory & GDB already exist at: ", full_gdb_path)

Directory & GDB already exist at:  C:\gitFiles\minnesota-bmsb-analysis\data\gdb\bmsb_analysis.gdb


In [3]:
# Set up Path Vars of Downloaded Datasets
cities_shp = r"C:\gitFiles\minnesota-bmsb-analysis\data\cities\city_township_unorg.shp"
elevation_gdb_rast = r"C:\gitFiles\minnesota-bmsb-analysis\data\elevation\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"
landcover_tiff = r"C:\gitFiles\minnesota-bmsb-analysis\data\landcover\NLCD_2019_Land_Cover.tif"
observations_csv = r"C:\gitFiles\minnesota-bmsb-analysis\data\observations\eddmaps_bmsb.csv"

### Rasters

In [4]:
# Load Elevation to GDB
arcpy.management.CopyRaster(elevation_gdb_rast, os.path.join(full_gdb_path, "elevation"))

In [5]:
# Load LC to GDB
arcpy.management.CopyRaster(landcover_tiff, os.path.join(full_gdb_path, "landcover"))

In [6]:
# Reclassify Landcover to Urban, Ag, Natural
reclassified_lc = arcpy.sa.Reclassify(
    in_raster=os.path.join(full_gdb_path, "landcover"),
    reclass_field="NLCD_Land",
    remap="Unclassified NODATA;'Open Water' 3;'Developed, Open Space' 1;'Developed, Low Intensity' 1;'Developed, Medium Intensity' 1;'Developed, High Intensity' 1;'Barren Land' 3;'Deciduous Forest' 3;'Evergreen Forest' 3;'Mixed Forest' 3;Shrub/Scrub 3;Herbaceous 3;Hay/Pasture 2;'Cultivated Crops' 2;'Woody Wetlands' 3;'Emergent Herbaceous Wetlands' 3",
    missing_values="DATA"
)

reclassified_lc.save(os.path.join(full_gdb_path, "reclass_lc"))

### Cities

In [7]:
# Load Cities to GDB
arcpy.conversion.ExportFeatures(
    cities_shp,
    os.path.join(full_gdb_path, "cities"),
    where_clause="CTU_CLASS = 'CITY'",
)

### Weather

In [8]:
# Extract & Transform Weather
weather_df = WeatherLoader.multi_month([4, 5, 6, 7, 8, 9], 2022)

# Display - WX Vals are the Mean Daily Values (i.e., Mean Max Temp)
weather_df

,station,name,x,y,max_tmpf,min_tmpf,precip
station,,,,,,,
MN001,MN001,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331,72.580797,52.340001,0.000511
MN002,MN002,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800,70.530819,50.808522,0.000273
MN003,MN003,Little Chicago I-35 Mile Post 70,-93.292427,44.478500,72.223605,50.588194,0.000164
MN004,MN004,Rush City I-35 Mile Post 157,-92.992752,45.642921,70.462952,49.659673,0.000273
MN005,MN005,Rutledge I-35 Mile Post 198,-92.838562,46.212570,72.076065,46.662624,0.000000
...,...,...,...,...,...,...,...
MN158,MN158,U.S.75 - Canby - MP 84.0 MN US MNDOT,-96.276932,44.674171,75.924734,55.797897,0.000000
MN159,MN159,U.S.12 - Atwater - MP 85.4 MN US MNDOT,-94.811760,45.139050,72.457368,55.660526,0.000000
MN160,MN160,U.S.14 - Florence - MP 21.2 MN US MNDOT,-96.046341,44.240311,74.513158,55.636839,0.000000


In [9]:
# Load to GDB
WeatherLoader.load(full_gdb_path, "WxAprToSep2022", weather_df)

### BMSB Observations

In [4]:
# Extract & Transform Observations
observations = ObservationLoader(observations_csv)

observations.df.head()

c:\gitFiles\minnesota-bmsb-analysis\notebooks\..\bmsb\etl.py:225: DtypeWarning: Columns (15,17,23,26,27,30,31,32,33,34,35,38,39,40,41,42,45,47,49,51,53,54,55,57,62,63,66,69,70,71,72,73,74,78) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(csv, encoding="unicode_escape")


,objectid,reporter,RecOwner,SciName,ComName,Nativity,OccStatus,Status,ObsDate,DateEnt,...,TreatComm,Reference,Locality,Comments,ReviewDate,Reviewer,VerifyMthd,Verified,IDCred,ReviewComm
0,2482976,Virginia Brown Marmorated Stink Bug Monitoring...,State Agency,Halyomorpha halys,brown marmorated stink bug,Introduced,Undetected,Negative,09-06-12,10-26-12,...,NaN,NaN,NaN,"""All fields surveyed in a county combined, ave...",NaN,NaN,NaN,Verified,NaN,NaN
1,2482977,Virginia Brown Marmorated Stink Bug Monitoring...,State Agency,Halyomorpha halys,brown marmorated stink bug,Introduced,Detected,Positive,09-16-12,10-26-12,...,NaN,NaN,NaN,"""All fields surveyed in a county combined, ave...",NaN,NaN,NaN,Verified,NaN,NaN
2,2482978,Virginia Brown Marmorated Stink Bug Monitoring...,State Agency,Halyomorpha halys,brown marmorated stink bug,Introduced,Detected,Positive,08-03-12,10-26-12,...,NaN,NaN,NaN,"""All fields surveyed in a county combined, ave...",NaN,NaN,NaN,Verified,NaN,NaN
3,2482979,Virginia Brown Marmorated Stink Bug Monitoring...,State Agency,Halyomorpha halys,brown marmorated stink bug,Introduced,Detected,Positive,08-10-12,10-26-12,...,NaN,NaN,NaN,"""All fields surveyed in a county combined, ave...",NaN,NaN,NaN,Verified,NaN,NaN
4,2482980,Virginia Brown Marmorated Stink Bug Monitoring...,State Agency,Halyomorpha halys,brown marmorated stink bug,Introduced,Detected,Positive,08-17-12,10-26-12,...,NaN,NaN,NaN,"""All fields surveyed in a county combined, ave...",NaN,NaN,NaN,Verified,NaN,NaN


In [7]:
# Transform Observations
observations.transform()

observations.df

,objectid,ObsDate,Latitude,Longitude
965,3047914,2012-09-27,45.09834,-93.31717
982,3047931,2012-09-22,46.86453,-96.76807
1035,3047984,2012-06-27,43.00840,-90.18980
1349,3048298,2011-11-16,43.67082,-92.94843
1513,3048462,2011-08-25,45.40731,-93.24103
...,...,...,...,...
95424,11292427,2023-01-13,44.88777,-92.97591
95425,11292428,2023-01-13,44.32806,-93.96222
95426,11292430,2023-01-13,44.32806,-93.96222
95427,11292431,2023-01-13,44.97945,-93.45546


In [8]:
# Load to GDB
observations.load(full_gdb_path, "BMSB_Obs")

### Aggregation

In [4]:
# Aggregate Landcover to Cities
arcpy.sa.ZonalHistogram(
    os.path.join(full_gdb_path, "cities"),
    "FEATURE_NA",
    os.path.join(full_gdb_path, "reclass_lc"),
    os.path.join(full_gdb_path, "agg_landcover"),
    zones_as_rows="ZONES_AS_ROWS"
)

<geoprocessing server result object at 0x1f0f2ea3210>

In [5]:
# Aggregate Elevation to Cities
arcpy.ia.ZonalStatisticsAsTable(
    os.path.join(full_gdb_path, "cities"),
    "FEATURE_NA",
    os.path.join(full_gdb_path, "elevation"),
    os.path.join(full_gdb_path, "agg_elevation"),
    ignore_nodata="DATA",
    statistics_type="ALL"
)

<geoprocessing server result object at 0x1f0f2e9a2a0>

In [6]:
# Aggregate Observation Counts to Cities
arcpy.analysis.SummarizeWithin(
    os.path.join(full_gdb_path, "cities"),
    os.path.join(full_gdb_path, "BMSB_Obs"),
    os.path.join(full_gdb_path, "agg_observations"),
    keep_all_polygons="KEEP_ALL",
    sum_fields=None,
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)

<Result 'C:\\gitFiles\\minnesota-bmsb-analysis\\data\\gdb\\bmsb_analysis.gdb\\agg_observations'>

In [7]:
# Join Nearest Weather Data to Cities
arcpy.analysis.SpatialJoin(
    os.path.join(full_gdb_path, "cities"),
    os.path.join(full_gdb_path, "WxAprToSep2022"),
    os.path.join(full_gdb_path, "agg_weather"),
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="CLOSEST",
    search_radius=None,
    distance_field_name=""
)

<Result 'C:\\gitFiles\\minnesota-bmsb-analysis\\data\\gdb\\bmsb_analysis.gdb\\agg_weather'>

In [13]:
# Join Aggregated Data back to Cities
joined = arcpy.management.AddJoin(
    os.path.join(full_gdb_path, "cities"),
    "FEATURE_NA",
    os.path.join(full_gdb_path, "agg_elevation"),
    "FEATURE_NA"
)

joined = arcpy.management.AddJoin(
    joined,
    "FEATURE_NA",
    os.path.join(full_gdb_path, "agg_weather"),
    "FEATURE_NA"
)

joined = arcpy.management.AddJoin(
    joined,
    "FEATURE_NA",
    os.path.join(full_gdb_path, "agg_observations"),
    "FEATURE_NA"
)

joined = arcpy.management.AddJoin(
    joined,
    "FEATURE_NA",
    os.path.join(full_gdb_path, "agg_landcover"),
    "LABEL"
)

# Copy Joined FC 
arcpy.management.CopyFeatures(
    joined,
    os.path.join(full_gdb_path, "cities_attributed")
)

<Result 'C:\\gitFiles\\minnesota-bmsb-analysis\\data\\gdb\\bmsb_analysis.gdb\\cities_attributed'>

In [16]:
# Clean Up Joined Dataset
arcpy.management.DeleteField(
    os.path.join(full_gdb_path, "cities_attributed"),
    "cities_GNIS_FEATU;cities_FEATURE_NA;cities_COUNTY_NAM;cities_POPULATION;cities_SHAPE_Leng;agg_elevation_RANGE;agg_elevation_MEAN;agg_elevation_STD;agg_elevation_MEDIAN;agg_weather_Join_Count;agg_weather_max_tmpf;agg_weather_min_tmpf;agg_weather_precip;agg_observations_Point_Count;agg_landcover_CLASS_1;agg_landcover_CLASS_2;agg_landcover_CLASS_3",
    "KEEP_FIELDS"
)

<Result 'C:\\gitFiles\\minnesota-bmsb-analysis\\data\\gdb\\bmsb_analysis.gdb\\cities_attributed'>

In [19]:
# Display as SEDF
pd.DataFrame.spatial.from_featureclass(os.path.join(full_gdb_path, "cities_attributed")).head()

,OBJECTID,cities_GNIS_FEATU,cities_FEATURE_NA,cities_COUNTY_NAM,cities_POPULATION,cities_SHAPE_Leng,agg_elevation_RANGE,agg_elevation_MEAN,agg_elevation_STD,agg_elevation_MEDIAN,agg_weather_Join_Count,agg_weather_max_tmpf,agg_weather_min_tmpf,agg_weather_precip,agg_observations_Point_Count,agg_landcover_CLASS_1,agg_landcover_CLASS_2,agg_landcover_CLASS_3,SHAPE
0,1,2394210,Bluffton,Otter Tail,208,12870.280762,84,1347.493968,21.954257,1353,1,69.002294,49.479673,0.000164,0,794,4350,2728,"{""rings"": [[[329659.49899999984, 5147112.93899..."
1,2,2396539,Sartell,Stearns,17412,88231.299454,120,1047.756071,16.313259,1047,1,69.840329,50.023607,0.000492,0,19568,7118,3978,"{""rings"": [[[401466.0499999998, 5050916.149], ..."
2,3,2393505,Cambridge,Isanti,9862,65438.361367,85,949.380606,17.984951,955,1,70.462952,49.659673,0.000273,1,12836,4015,5822,"{""rings"": [[[479383.5999999996, 5046640.25], [..."
3,4,2397203,Waseca,Waseca,9221,31106.933645,56,1136.962191,12.658397,1135,1,72.184263,52.943936,0.000383,0,9359,1836,4548,"{""rings"": [[[456193.8666000003, 4880132.7445],..."
4,5,2395562,La Crescent,Houston,5272,24252.756330,584,725.382273,128.798479,679,1,74.276315,58.962108,0.000000,0,5603,536,4860,"{""rings"": [[[633758.4579999996, 4853600.790999..."
